### MNIST Handwritten Digit Recognizer With Live Video Feed
<pre>
Authors            : Rajendra Bhagroo  | Christopher Guevarra
Emails             : Rbhagroo@Nyit.edu | Cguevarr@nyit.edu

Course Name        : Artificial Intelligence 1
Course Number      : CSCI 355 - W01D
Course Instructor  : Maherukh Akhtar

Date Due           : 12/10/18
Date Last Modified : 11/18/18
Kernel             : Python 3.6.6
</pre>

In [ ]:
# Install pip packages for the current Jupyter kernel
from sys import executable

!{executable} -m pip install --upgrade pip
!{executable} -m pip install matplotlib
!{executable} -m pip install keras

# Python 3.6 Compatable: https://www.tensorflow.org/install/pip
!{executable} -m pip install --upgrade tensorflow

In [ ]:
'''
Sets matplotlib's backend engine to display plots within current Jupyter kernel
Sets Tensorflow as backend engine
'''
%matplotlib inline
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# https://keras.io/datasets/#mnist-database-of-handwritten-digits
# To implement different dataset see: http://yann.lecun.com/exdb/mnist/
from keras.datasets import mnist

# Keras is a high level API that will use TensorFlow for its backend engine
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [ ]:
# Load Data
'''
Dataset Info:
    28x28 greyscale images [Digits 0-9]
    Training Set: 60,000 images
    Testing Set: 10,000 images
    X_train, X_test: uint8 array of grayscale image data with shape (num_samples, 28, 28)
    y_train, y_test: uint8 array of digit labels (integers in range 0-9) with shape (num_samples,)
'''
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# Preprocess Data
'''
Inspecting the data, we can see that we are working with a numpy array with shape (60000, 28, 28)

Shape:
    Number Of Samples: 60,000
    Rows    [pixels] : 28
    Columns [pixels] : 28
'''

print(f'''
Before Manipulation
    X_train class type: {type(X_train)}
    X_train data type: {X_train.dtype}
    X_train shape: {X_train.shape}
''')


# Reshape Data
'''
Reshape numpy array from 3D to 4D, resembling [samples][channels][width][height]
1 channel = greyscale images, we would use 3 channels for RGB colored images.

We will also change the datatype from uint8 to float32 to work with decimal range [0-1]
'''

X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

print(f'''
After Manipulation
    X_train class type: {type(X_train)}
    X_train data type: {X_train.dtype}
    X_train shape: {X_train.shape}
''')

In [ ]:
# Normalize Data 
# Input images are represented by 8-bits, 0-255 range [RGB]. lets change the range to 0-1 [Greyscale]

print(f'''
Un-Normalized Array [RGB: 0-255]\n
{X_train[0][0][8]}
''')

X_train /= 255
X_test /= 255

print(f'''
Normalized Array [Greyscale: 0-1]\n
{X_train[0][0][8]}
''')


# One-Hot Encode Data
'''
Encode using one-of-K scheme. Converts raw values into numerical form to feed into model
Binary vector representation of categorical data
Example:                  0123456789
         Raw Digit [4] = [0001000000]
         Raw Digit [9] = [0000000010]
'''

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(f"Number Of Classes: {y_test.shape[1]}")

In [ ]:
# Plot Data

'''
Using Matplotlib to plot the first 5 images in the test set, labels included in one-hot form
Remember This Example?

Example:                  0123456789
         Raw Digit [4] = [0001000000]
         Raw Digit [9] = [0000000010]
'''
for i in range(5):
    image = X_test[i]
    image = np.array(image, dtype='float32')
    pixels = image.reshape((28, 28))
    plt.title(f'One-Hot Encoded Label\n {y_test[i]}')
    plt.imshow(pixels, cmap='gray')
    plt.show()

In [ ]:
# Build Model
# SHUFFLE DATA! 
# Evaluate Model
# Save Model

In [ ]:
# OpenCV